#### Notebook to feature engineer fraction of time temperature was above 90th percentile

In [1]:
#conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 15)

In [2]:
PATH_1=os.path.join('..','..','..','..','Weather_Data','CAMS','2014','2014_multi_level_26_variables.nc')
ml_26_variables = xr.open_dataset(PATH_1) ## multilevel 26 variables
ml_26_variables = ml_26_variables.squeeze(dim="model_level") ## remove the model_level dimension
ml_26_variables=ml_26_variables.drop_vars('model_level') ## the dropped dimension becomes a column, so remove that
ml_26_variables['longitude'] = ml_26_variables['longitude']-360 ## necessary because of longitude data in multi-level case
temp=ml_26_variables['t'] ## get only the temperature variable
temp

<xarray.DataArray 't' (valid_time: 2920, latitude: 34, longitude: 78)> Size: 31MB
[7743840 values with dtype=float32]
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 23kB 2014-01-01 ... 2014-12-31T21...
  * latitude    (latitude) float64 272B 49.25 48.5 47.75 ... 26.0 25.25 24.5
  * longitude   (longitude) float64 624B -124.8 -124.1 -123.3 ... -67.85 -67.1
Attributes: (12/31)
    GRIB_paramId:                             130
    GRIB_dataType:                            an
    GRIB_numberOfPoints:                      2652
    GRIB_typeOfLevel:                         hybrid
    GRIB_stepUnits:                           1
    GRIB_stepType:                            instant
    ...                                       ...
    GRIB_pv:                                  [0.00000000e+00 2.00000000e+01 ...
    GRIB_shortName:                           t
    GRIB_units:                               K
    long_name:                                Temperature
    units:                                    K
    standard_name:                            air_temperature

In [3]:
temp.to_dataframe()

t
valid_time          latitude longitude                                
2014-01-01 00:00:00 49.25    -124.849999999999994  278.681884765625000
                             -124.099999999999994  278.471923828125000
                             -123.349999999999994  278.087158203125000
                             -122.599999999999994  276.647705078125000
                             -121.849999999999994  274.812744140625000
...                                                                ...
2014-12-31 21:00:00 24.50    -70.100000000000023   297.341094970703125
                             -69.350000000000023   297.615509033203125
                             -68.600000000000023   297.712188720703125
                             -67.850000000000023   297.808868408203125
                             -67.100000000000023   297.796173095703125

[7743840 rows x 1 columns]

In [4]:
PATH_2=os.path.join('..','CAMS_79_variables_2014.pkl')
get_data = pd.read_pickle(PATH_2) ## load the data with all variables
get_data.head()

,NAME,fips,u10,v10,d2m,t2m,bcaod550,duaod550,lsm,msl,omaod550,pm1,pm2p5,pm10,ssaod550,suaod550,z,sp,aod469,aod550,aod670,aod865,aod1240,tcco,tc_c2h6,tchcho,tc_h2o2,tc_oh,tc_c5h8,tc_ch4,tc_hno3,tcno2,tc_no,gtco3,tc_pan,tc_c3h8,tcso2,tcwv,co,aermr04,aermr05,aermr06,c2h6,hcho,h2o2,aermr09,aermr07,aermr10,aermr08,oh,c5h8,hno3,no2,no,go3,pan,c3h8,aermr01,aermr02,aermr03,q,aermr11,so2,t,lai_hv,lai_lv,asn,sd,tcw,aermssdus,aermssdum,aermssdul,aermssbchphil,aermssomhphil,aermssbchphob,aermssomhphob,aermsssss,aermssssm,aermssssl,aermsssu,aermssso2
0,Autauga,1001,0.251940518536598,0.044441910528852,284.069722028797401,290.566949063257937,0.006760782718137,0.012491468873007,0.986987539115746,101843.750267399300355,0.073715149002067,0.000000011580616,0.00000001503748,0.000000020553803,0.0079086762121,0.065185196072414,1312.642868868908636,100247.763403065648163,0.201122994597222,0.166061144100417,0.128985708789875,0.091595399447572,0.054429319711792,0.000856227314981,0.000003345425686,0.00000404820085,0.000008576059899,0.000000004861008,0.000003479987232,0.009955392815841,0.000004644541922,0.000002966720971,0.000000431552958,0.006382678230507,0.000008609472475,0.000000619108319,0.000002612182458,25.418044775170156,0.000000177021834,0.000000000744561,0.000000001298071,0.000000000470626,0.000000000602238,0.000000003909151,0.00000000081518,0.000000000227406,0.00000000759163,0.0000000002757,0.000000004239988,0.000000000000021,0.000000019969136,0.000000000784627,0.000000006908364,0.000000000457291,0.000000056623125,0.000000002894,0.000000000251805,0.000000000020971,0.000000001696678,0.000000000193799,0.00949899018259,0.000000002223097,0.000000005090805,290.555313103630283,3.906738033237082,2.625992421689393,0.879287593711123,0.002205393569465,25.574830776023337,0.000002408201269,0.00000494122011,0.000004178808052,0.000000130280661,0.000002277404793,0.000000585155998,0.000012511550778,0.000000197818194,0.000016061330896,0.000000781181336,0.000006899787827,0.000001557584619
839,Baldwin,1003,-0.172408265707899,-0.102088212710376,287.97745678395296,292.490239579616116,0.006844172411079,0.016102232331852,0.52644905611773,101823.391639321635012,0.071606517721315,0.000000009050574,0.000000012637138,0.000000017356516,0.011365301938302,0.068701375618045,217.270177173356785,101558.322291202101042,0.210035162562767,0.174619567381612,0.137177839172166,0.099638838627951,0.061434673645138,0.000833837234461,0.000003261504084,0.000003148197306,0.000009448850952,0.000000004928978,0.000001271770121,0.010086562293025,0.000004832350031,0.000002568330316,0.000000397582642,0.006260362030456,0.00000742943947,0.000000566340969,0.000001980695422,28.475489446688151,0.000000150363063,0.000000000805996,0.000000001355808,0.000000000468977,0.000000000527444,0.000000002309754,0.000000000913015,0.000000000199198,0.000000005679923,0.000000000217788,0.000000002784373,0.000000000000035,0.000000006704661,0.000000000973215,0.000000005010373,0.000000000330287,0.000000062069038,0.000000002035879,0.000000000206347,0.000000000059863,0.000000004958306,0.000000002829293,0.011598901616715,0.000000002008615,0.000000002759892,292.461674529969002,3.035802087951026,1.908562090441901,0.879852713016083,0.000728902648398,28.599052012856351,0.000003123130032,0.000006433563398,0.000005100825474,0.000000124840699,0.00000193107874,0.00000059970705,0.000012393948455,0.000000274634833,0.000022396692749,0.000002022298193,0.00000734001244,0.000000909139999
1,Barbour,1005,0.249173664115031,-0.02693361599547,284.585892231687012,291.27818064079122,0.006901586971228,0.012974777608821,0.989715878565123,101836.77728783992643,0.074392240529005,0.000000012253607,0.000000015890748,0.000000021660226,0.008871928442056,0.065021258307229,1083.188991193836955,100519.980237769370433,0.203307210897243,0.168161829986728,0.130986925824832,0.093492914073556,0.056022341776388,0.000849962583106,0.000003313447075,0.00000390026062,0.000008877697132,0.00000000486878,0.000002948520409,0.009983542212264,0.000004698995498,0.0000

In [5]:
percentile_75 = get_data['t'].quantile(0.90) ## get temperature data
percentile_75
threshold=percentile_75

In [6]:
SHAPE_PATH=os.path.join('..','..','..','..','Shapefiles','county_shapefiles','2014_county_shapefile','cb_2014_us_county_500k.shp')
county_gdf = gpd.read_file(SHAPE_PATH)
county_gdf

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,01,001,00161526,0500000US01001,01001,Autauga,06,1539584444,25773561,"POLYGON Z ((-86.92120 32.65754 0.00000, -86.92..."
1,01,005,00161528,0500000US01005,01005,Barbour,06,2291820953,50864677,"POLYGON Z ((-85.74803 31.61918 0.00000, -85.74..."
2,01,023,00161537,0500000US01023,01023,Choctaw,06,2365954803,19059247,"POLYGON Z ((-88.47323 31.89386 0.00000, -88.46..."
3,01,033,00161542,0500000US01033,01033,Colbert,06,1534878355,80029923,"POLYGON Z ((-88.13925 34.58779 0.00000, -88.13..."
4,01,047,00161549,0500000US01047,01047,Dallas,06,2534807336,39134779,"POLYGON Z ((-87.47308 32.30761 0.00000, -87.42..."
...,...,...,...,...,...,...,...,...,...,...
3228,17,015,00424209,0500000US17015,17015,Carroll,06,1152707810,55841451,"POLYGON Z ((-90.31795 42.19391 0.00000, -90.22..."
3229,31,131,00835887,0500000US31131,31131,Otoe,06,1594475777,8775373,"POLYGON Z ((-96.46384 40.69696 0.00000, -96.46..."
3230,33,011,00873179,0500000US33011,33011,Hillsborough,06,2269220216,41604851,"POLYGON Z ((-72.06124 42.96584 0.00000, -72.05..."
3231,39,027,01074026,0500000US39027,39027,Clinton,06,1058488036,9350863,"POLYGON Z ((-84.00373 39.28847 0.00000, -84.00..."


In [7]:
geometry_column=county_gdf['geometry']
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x                   y
0    0   -86.921195999999995  32.657541999999999
     1   -86.920351999999994  32.658563000000001
     2   -86.920411000000001  32.660077000000001
     3   -86.917594999999992  32.664169000000001
     4   -86.914609999999996  32.664355000000000
...                      ...                 ...
3232 120 -83.155472000000003  39.511775999999998
     121 -83.165610999999998  39.512189999999997
     122 -83.224839000000003  39.514578999999998
     123 -83.266736999999992  39.516248999999995
     124 -83.264480999999989  39.540672000000001

[1048659 rows x 2 columns]

In [8]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a 100 or less latitude and longitude pair.
        And then to find the fraction of time, PM2.5 was above EPA standard
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            pm_above_df: Dataframe consisting the values of the variables interpolated in the county.
    '''   
    
    
    longitude= lat_lon.loc[(var), 'x']  # get the longitude
    extract_val=len(longitude) ## find number of longitude that a shapfile has

    ## if number of longitude or latitude is less than 100, that many lat-lon pairs will be extracted, if not, a 100 values
    if extract_val < 100:
        extract_val=extract_val
    else:
        extract_val=100

    longitude=longitude[:extract_val]  ## extract first 100 values or values less than 100
    latitude=lat_lon.loc[(var),'y']    ## get the latitude values
    latitude=latitude[:extract_val]    ## extract first 100 values or values less than 100

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    ## find the corresponding values of the variables in the finer grid

    year_avg_finer= temp.interp(longitude=lon_list, latitude=lat_list)
    
## get only the 100 (or less) pair of latitude and longitude from the lat and lon list from the 100,000 (or less) rows
    get_vals=[]

    for i in range(0, extract_val):
        row=year_avg_finer.isel(latitude=[i], longitude=[i])
        row_df=row.to_dataframe()
        get_vals.append(row_df)
    
    initial_df=pd.concat(get_vals)
    summary = initial_df.groupby("valid_time").mean(["latitude", "longitude"])
    df=summary

    total_rows=len(df)

    c=0
    for i in range(0, total_rows):
        if df['t'].values[i] > threshold:
            c=c+1

## convert number to a fraction
    frac_time=(c/total_rows)*100
    pm_above_df=pd.DataFrame({'Temp above percentile':[frac_time]})


    return pm_above_df

In [9]:
len(county_gdf)

3233

In [10]:
%%time

## get the values for all counties

df_list_below=[]

for i in range(0,len(county_gdf)): ## loop for the all list of counties
    try:
        df_list_below.append(single_county(i))
    except:
        pass

CPU times: total: 1h 19min 47s
Wall time: 1h 32min 37s


In [11]:
concatenate_df=pd.concat(df_list_below)
final_df=concatenate_df.reset_index(drop=True)
final_df

,Temp above percentile
0,51.027397260273979
1,52.260273972602747
2,52.705479452054796
3,45.239726027397261
4,51.506849315068493
...,...
3228,28.356164383561644
3229,34.383561643835613
3230,20.068493150684933
3231,31.712328767123289


In [12]:
## lets first create a fips column in the original shape file

county_gdf['fips']=county_gdf['STATEFP'] + county_gdf['COUNTYFP']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

county_gdf = county_gdf.drop(['STATEFP','COUNTYFP','COUNTYNS','AFFGEOID','GEOID','LSAD','ALAND','AWATER'], axis=1)
county_gdf

,NAME,geometry,fips
0,Autauga,"POLYGON Z ((-86.92120 32.65754 0.00000, -86.92...",1001
1,Barbour,"POLYGON Z ((-85.74803 31.61918 0.00000, -85.74...",1005
2,Choctaw,"POLYGON Z ((-88.47323 31.89386 0.00000, -88.46...",1023
3,Colbert,"POLYGON Z ((-88.13925 34.58779 0.00000, -88.13...",1033
4,Dallas,"POLYGON Z ((-87.47308 32.30761 0.00000, -87.42...",1047
...,...,...,...
3228,Carroll,"POLYGON Z ((-90.31795 42.19391 0.00000, -90.22...",17015
3229,Otoe,"POLYGON Z ((-96.46384 40.69696 0.00000, -96.46...",31131
3230,Hillsborough,"POLYGON Z ((-72.06124 42.96584 0.00000, -72.05...",33011
3231,Clinton,"POLYGON Z ((-84.00373 39.28847 0.00000, -84.00...",39027


In [13]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,Temp above percentile
0,Autauga,"POLYGON Z ((-86.92120 32.65754 0.00000, -86.92...",1001,51.027397260273979
1,Barbour,"POLYGON Z ((-85.74803 31.61918 0.00000, -85.74...",1005,52.260273972602747
2,Choctaw,"POLYGON Z ((-88.47323 31.89386 0.00000, -88.46...",1023,52.705479452054796
3,Colbert,"POLYGON Z ((-88.13925 34.58779 0.00000, -88.13...",1033,45.239726027397261
4,Dallas,"POLYGON Z ((-87.47308 32.30761 0.00000, -87.42...",1047,51.506849315068493
...,...,...,...,...
3228,Carroll,"POLYGON Z ((-90.31795 42.19391 0.00000, -90.22...",17015,28.356164383561644
3229,Otoe,"POLYGON Z ((-96.46384 40.69696 0.00000, -96.46...",31131,34.383561643835613
3230,Hillsborough,"POLYGON Z ((-72.06124 42.96584 0.00000, -72.05...",33011,20.068493150684933
3231,Clinton,"POLYGON Z ((-84.00373 39.28847 0.00000, -84.00...",39027,31.712328767123289


In [14]:
county_df=merged_df.dropna()
county_df.head()

,NAME,geometry,fips,Temp above percentile
0,Autauga,"POLYGON Z ((-86.92120 32.65754 0.00000, -86.92...",1001,51.027397260273979
1,Barbour,"POLYGON Z ((-85.74803 31.61918 0.00000, -85.74...",1005,52.260273972602747
2,Choctaw,"POLYGON Z ((-88.47323 31.89386 0.00000, -88.46...",1023,52.705479452054796
3,Colbert,"POLYGON Z ((-88.13925 34.58779 0.00000, -88.13...",1033,45.239726027397261
4,Dallas,"POLYGON Z ((-87.47308 32.30761 0.00000, -87.42...",1047,51.506849315068493


In [15]:
## convert into pandas dataframe without the geometry column
pd.options.display.float_format = '{:.15f}'.format ## see 15 decimal places of the numbers
county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var=county_var.rename(columns={'PM2.5 above threshold':'PM2.5 above percentile'})
county_var

,NAME,fips,Temp above percentile
0,Autauga,1001,51.027397260273979
1,Barbour,1005,52.260273972602747
2,Choctaw,1023,52.705479452054796
3,Colbert,1033,45.239726027397261
4,Dallas,1047,51.506849315068493
...,...,...,...
3228,Carroll,17015,28.356164383561644
3229,Otoe,31131,34.383561643835613
3230,Hillsborough,33011,20.068493150684933
3231,Clinton,39027,31.712328767123289


In [16]:
county_var.to_pickle('temp_above_percentile.pkl')